In [1]:
import numpy as np
import itertools

In [2]:
with open('../code/script-cnn.sh', 'a') as the_file:
    the_file.write("#!bin/bash \n")
    for dataset in [2]:
        for fold_num in range(5):
            for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
                for lr in [0.001, 0.01, 0.1, 1]:
                    for iterations in [2000, 4000, 6000, 8000]:
                        line = "python cnn-individual.py {} {} {} {} {} \n".format(dataset, fold_num, appliance, lr, iterations)
                        the_file.write(line)

In [5]:
orders = np.load("../code/baseline/rnn-greedy2-orders.npy").item()

In [10]:
best = np.load("../code/baseline/best_param_nested_rnn.npy").item()

In [2]:
k = list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw']))[0]

In [21]:
" ".join(str(x) for x in k)

'hvac fridge dr dw mw'

In [11]:
gid = 0
srv = 1
c = 0
dataset = 2
for fold_num in range(1, 5):
    for lr in [0.001, 0.01, 0.1]:
        for iters in [20000]:
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py {} {} {} 0 {} {} \n".format(gid, dataset, lr, iters, fold_num, o))
                c+=1

                with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
                    the_file.write("#!bin/bash \n")
                    the_file.write(line)

                if c%120 == 0:
                    gid += 1
                if gid == 3:
                    gid = 0
                    srv += 1
print(c, srv, gid)

1440 5 0


In [5]:
line

'CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 1 0.1 20000 0 4 mw dw dr fridge hvac \n'

In [59]:
for i in range(1, 7):
    for j in range(4):
        print("nohup sh 0{}{}.sh &> 0{}{}.out &".format(i, j, i, j))

nohup sh 010.sh &> 010.out &
nohup sh 011.sh &> 011.out &
nohup sh 012.sh &> 012.out &
nohup sh 013.sh &> 013.out &
nohup sh 020.sh &> 020.out &
nohup sh 021.sh &> 021.out &
nohup sh 022.sh &> 022.out &
nohup sh 023.sh &> 023.out &
nohup sh 030.sh &> 030.out &
nohup sh 031.sh &> 031.out &
nohup sh 032.sh &> 032.out &
nohup sh 033.sh &> 033.out &
nohup sh 040.sh &> 040.out &
nohup sh 041.sh &> 041.out &
nohup sh 042.sh &> 042.out &
nohup sh 043.sh &> 043.out &
nohup sh 050.sh &> 050.out &
nohup sh 051.sh &> 051.out &
nohup sh 052.sh &> 052.out &
nohup sh 053.sh &> 053.out &
nohup sh 060.sh &> 060.out &
nohup sh 061.sh &> 061.out &
nohup sh 062.sh &> 062.out &
nohup sh 063.sh &> 063.out &


In [19]:
gid = 0
srv = 1
c = 0


for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
    o = " ".join(str(x) for x in order)
    line = ("CUDA_VISIBLE_DEVICES={} python rnn-cnn-tree.py 1 GRU 20 1 True 0.01 3000 0 0 {}\n".format(gid, o))
    c+=1

    with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
        the_file.write("#!bin/bash \n")
        the_file.write(line)

    if c%60 == 0:
        gid += 1
    if gid == 4:
        gid = 0
        srv += 1
            
print(c)


120


In [17]:
print(gid, srv)

0 1


In [3]:
gid = 0
srv = 1
c = 0



for fold_num in range(5):
    for cell_type in ['GRU', 'LSTM', 'RNN']:
        for hidden_size in [20, 50, 100]:
            for num_layers in [1, 2, 3, 4]:
                for bidirectional in [True, False]:
                    for lr in [0.001, 0.01, 0.1, 1]:
                        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'])):
                            o = " ".join(str(x) for x in order)
                            line = ("CUDA_VISIBLE_DEVICES={} python rnn-nested-cv.py 2 {} {} {} {} {} 3000 0 {} {}\n".format(gid, cell_type, hidden_size, num_layers, bidirectional, lr, fold_num, o))
                            c+=1
 
#                             with open('../code/0{}{}.sh'.format(srv, gid), 'a') as the_file:
#                                 the_file.write("#!bin/bash \n")
#                                 the_file.write(line)

                            if c%7200 == 0:
                                gid += 1
                            if gid == 4:
                                gid = 0
                                srv += 1

In [4]:
c

172800

In [36]:
app_list = ['hvac', 'fridge', 'dr', 'dw', 'mw']

In [41]:
" ".join(order)

'hvac fridge'

In [43]:
order = ['hvac', 'fridge']
fold_num = 0
for app in app_list:
    if app in order:
        continue
    else:
        print (app)
        
        print ("CUDA_VISIBLE_DEVICES={} python rnn-nested-cv.py {} 2 GRU 20 1 True 0.01 3000 0 {} {}".format(gid, fold_num, " ".join(order), app))

dr
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 2 GRU 20 1 True 0.01 3000 0 hvac fridge dr
dw
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 2 GRU 20 1 True 0.01 3000 0 hvac fridge dw
mw
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 2 GRU 20 1 True 0.01 3000 0 hvac fridge mw


In [54]:
import random

gid=0
for i in range(20):
    index = random.sample(list(np.arange(120)), 1)
#     print(index)
    order = list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw']))[index[0]]
    o = " ".join(str(x) for x in order)
    print ("CUDA_VISIBLE_DEVICES={} python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 {}".format(gid, o))
    gid+= 1
    if gid == 4:
        gid = 0
    

CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 fridge hvac dw mw dr
CUDA_VISIBLE_DEVICES=1 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 dr dw mw hvac fridge
CUDA_VISIBLE_DEVICES=2 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 mw hvac dr dw fridge
CUDA_VISIBLE_DEVICES=3 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 dw mw dr fridge hvac
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 fridge dw mw dr hvac
CUDA_VISIBLE_DEVICES=1 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 hvac fridge dr dw mw
CUDA_VISIBLE_DEVICES=2 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 hvac mw fridge dr dw
CUDA_VISIBLE_DEVICES=3 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 dr dw hvac fridge mw
CUDA_VISIBLE_DEVICES=0 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 mw dw dr hvac fridge
CUDA_VISIBLE_DEVICES=1 python rnn-nested-cv.py 0 1 GRU 50 1 True 0.1 2000 0.4 mw hvac dr dw fridge
CUDA_VISIB

In [13]:
for fold_num in range(5):
    for cell_type in ['GRU', 'LSTM', 'RNN']:
        for hidden_size in [20, 50, 100]:
            for num_layers in [1, 2, 3, 4]:
                for bidirectional in [True,False]:
                    for lr in [0.01, 0.1, 1.0]:
                        print("mkdir -p {}/{}/{}/{}/{}/{}/0.0".format(fold_num, cell_type, hidden_size, num_layers, bidirectional, lr))
                        print("mv *{}-1-{}-{}-{}-{}-{}* ./{}/{}/{}/{}/{}/{}/0.0".format(fold_num, cell_type,hidden_size,num_layers, bidirectional, lr, fold_num, cell_type,hidden_size,num_layers, bidirectional, lr))

mkdir -p 0/GRU/20/1/True/0.01/0.0
mv *0-1-GRU-20-1-True-0.01* ./0/GRU/20/1/True/0.01/0.0
mkdir -p 0/GRU/20/1/True/0.1/0.0
mv *0-1-GRU-20-1-True-0.1* ./0/GRU/20/1/True/0.1/0.0
mkdir -p 0/GRU/20/1/True/1.0/0.0
mv *0-1-GRU-20-1-True-1.0* ./0/GRU/20/1/True/1.0/0.0
mkdir -p 0/GRU/20/1/False/0.01/0.0
mv *0-1-GRU-20-1-False-0.01* ./0/GRU/20/1/False/0.01/0.0
mkdir -p 0/GRU/20/1/False/0.1/0.0
mv *0-1-GRU-20-1-False-0.1* ./0/GRU/20/1/False/0.1/0.0
mkdir -p 0/GRU/20/1/False/1.0/0.0
mv *0-1-GRU-20-1-False-1.0* ./0/GRU/20/1/False/1.0/0.0
mkdir -p 0/GRU/20/2/True/0.01/0.0
mv *0-1-GRU-20-2-True-0.01* ./0/GRU/20/2/True/0.01/0.0
mkdir -p 0/GRU/20/2/True/0.1/0.0
mv *0-1-GRU-20-2-True-0.1* ./0/GRU/20/2/True/0.1/0.0
mkdir -p 0/GRU/20/2/True/1.0/0.0
mv *0-1-GRU-20-2-True-1.0* ./0/GRU/20/2/True/1.0/0.0
mkdir -p 0/GRU/20/2/False/0.01/0.0
mv *0-1-GRU-20-2-False-0.01* ./0/GRU/20/2/False/0.01/0.0
mkdir -p 0/GRU/20/2/False/0.1/0.0
mv *0-1-GRU-20-2-False-0.1* ./0/GRU/20/2/False/0.1/0.0
mkdir -p 0/GRU/20/2/False/1